### Import Libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np
import tensorflow
from lib_file import lib_path
import yagmail
import pickle

### Model Loading

In [2]:
model = tensorflow.keras.models.load_model("models/ConvolutionalLongShortTermMemory_model.h5", compile=False)

In [3]:
with open(file="models/Scaler.pkl", mode="rb") as file:
    scaler = pickle.load(file=file)

In [4]:
with open(file="models/Important_Columns.pkl", mode="rb") as file:
    imp_cols = pickle.load(file=file)
imp_cols.remove('class')

In [5]:
IP_data = pd.read_csv("user_input/IP_data.csv")
IP_data.drop(IP_data.index, axis=0, inplace=True)
IP_data.to_csv("user_input/IP_data.csv", index=False)
IP_data = pd.read_csv("user_input/IP_data.csv")
IP_data

,Ip_Addresses


In [6]:
class_labels = ['BENIGN', 'DDos']

### User Input

In [22]:
test_data = pd.read_csv("user_input/sample_9.csv")
test_data.columns = [col.strip() for col in test_data.columns]
test_data.head()

,IP,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,28.224.224.255,80,10061307,5,0,30,0,6,6,6,0,0,0,0,0,2.98172,0.496953,2515326.75,5029323.587,10100000,1,10100000,2515326.75,5029323.587,10100000,1,0,0,0,0,0,0,0,0,0,100,0,0.496953,0,6,6,6,0,0,0,0,0,0,1,0,0,0,0,7.2,6,0,100,0,0,0,0,0,0,5,30,0,0,256,-1,4,20,1995,0,1995,1995,10100000,0,10100000,10100000


In [23]:
request_ip = str(test_data['IP'].values.tolist()[0])
print(request_ip)

28.224.224.255


In [24]:
test_data = test_data[imp_cols]
test_data

,Bwd Packet Length Mean,Avg Bwd Segment Size,Bwd Packet Length Max,Bwd Packet Length Std,Destination Port,URG Flag Count,Packet Length Mean,Average Packet Size,Packet Length Std,Min Packet Length,Max Packet Length,Packet Length Variance,min_seg_size_forward,Bwd Packet Length Min,Avg Fwd Segment Size,Fwd Packet Length Mean,Fwd Packet Length Max,Total Length of Fwd Packets,Subflow Fwd Bytes,Bwd IAT Total
0,0,0,0,0,80,0,6,7.2,0,6,6,0,20,0,6,6,6,30,30,0


In [25]:
scaled_input_=scaler.transform(test_data)
test_data=pd.DataFrame(data=scaled_input_,columns=imp_cols)
test_data.head()

,Bwd Packet Length Mean,Avg Bwd Segment Size,Bwd Packet Length Max,Bwd Packet Length Std,Destination Port,URG Flag Count,Packet Length Mean,Average Packet Size,Packet Length Std,Min Packet Length,Max Packet Length,Packet Length Variance,min_seg_size_forward,Bwd Packet Length Min,Avg Fwd Segment Size,Fwd Packet Length Mean,Fwd Packet Length Max,Total Length of Fwd Packets,Subflow Fwd Bytes,Bwd IAT Total
0,0.0,0.0,0.0,0.0,0.001221,0.0,0.003098,0.002848,0.0,0.017804,0.000514,0.0,0.384615,0.0,0.001552,0.001552,0.000514,0.000164,0.000164,0.0


### Model Prediction

In [26]:
model_prediction = model.predict(test_data.values, verbose=1)
class_label = np.argmax(model_prediction, axis=1)
class_name = class_labels[class_label[0]]
probability = f"{(model_prediction[0][class_label[0]])*100.0:.4f}%"

if class_name != 'BENIGN':
    user = yagmail.SMTP(user='[put your email here]',password='dohkkncralrhsycw')
    user.send(to='shreyaskatokar@gmail.com',subject='⚠️ALERT⚠️',contents=f"Intrusion request found from the IP: {request_ip}")
    print("Email sent sucessfully 📌")
    
    new_row = {'Ip_Addresses': request_ip}
    IP_data = IP_data.append(new_row, ignore_index=True)
    IP_data.to_csv("user_input/IP_data.csv", index=False)
    IP_data = pd.read_csv("user_input/IP_data.csv")

print(f"\nCLASS LABEL : {class_label}")
print(f"CLASS NAME : {class_name}")
print(f"PROBABILITY : {probability}\n")
IP_data.head(100)

1/1 [==============================] - 0s 58ms/step
Email sent sucessfully 📌

CLASS LABEL : [1]
CLASS NAME : DDos
PROBABILITY : 99.9393%



,Ip_Addresses
0,13.224.224.255
1,127.0.0.0.8888
2,28.224.224.255


---